In [8]:
import streamlit as st
import pandas as pd

from sklearn.metrics import classification_report

import findspark
findspark.init()

import sparknlp
import sparknlp

spark = sparknlp.start()
import pyspark
from pyspark.ml import Pipeline

from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
from pyspark.sql.types import StringType

In [13]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
     .setInputCols(["document"])\
     .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.load("./nbs/tmp_sentimentdl_model") \
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")

pipeline = Pipeline(
    stages = [
        document,
        use,
        sentimentdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]


In [14]:
def test_sentiment_func(test_input):

    test_sentence_df = spark.createDataFrame([test_input],StringType()).toDF("text")
    test_pred = pipeline.fit(test_sentence_df).transform(test_sentence_df)
    test_ans_df=test_pred.select("class.result").toPandas()
    print(test_ans_df)
text_input = input()
test_sentiment_func(text_input)

hello world
       result
0  [positive]
